In [1]:
import torch
import math
import matplotlib.pyplot as plt

import ModulesSalvage as MS
import SequentialSalvage as SS 
import OptimizerSalvage as OS

def generate_disc_set(nb_sample):
    
    data = torch.empty(nb_sample, 2).uniform_(0, 1)
    labels = data.sub(0.5).pow(2).sum(1).sub(1/(2*math.pi)).sign().add(1).div(2).long()
    
    data_out = data[torch.where(labels==1)]
    data_in = data[torch.where(labels==0)]
    
    plt.figure(figsize=(6,6))
    plt.scatter(data_out[:,0],data_out[:,1])
    plt.scatter(data_in[:,0], data_in[:,1])

    return data, labels

data, target = generate_disc_set(1000)
data1 = data[0:4]
target1 = target[0:4]
m1 = MS.Linear(2,25)
m2 = MS.Linear(25,25)
m3 = MS.Linear(25,25)
m4 = MS.Linear(25,2)
relu = MS.ReLu()
MSE = MS.MSE()

In [2]:
# Modules sequence
model = []
model = SS.Sequential(m1,relu,m2,relu,m4,relu)
model.zero_grad()
# loss criterion
MSE = MS.MSE()
cross = MS.CrossEntropyLoss()
for e in range(1000):
    #print("--------- EPOCH {} ---------- \n".format(e))
    #print("-------modelweights-------- \n",model.parameters[0][0])
    output = model(data)
    #print("-------modelweights AFTER FORWARD-------- \n",model.parameters[0][0])
    loss = MSE.forward(output,target)/1000
    print("LOSS :::",loss)
    dl_dx = MSE.backward(output,target)/1000
    #print("Grad(W) Before ZEROGRAD ------------\n",model.param()[0][1])
    model.zero_grad()
    #print("Grad(W) AFTER ZERO GRAD,Before BACKWARD ------------\n",model.param()[0][1])
    model.backward(dl_dx)
    sgd = OS.SGD(model.param(),eta=9e-2)
    #print("-------modelweights after BACKWARD----- \n",model.parameters[0][0])
    #print("Grad(W) AFTER BACKWARD----------- \n",model.param()[0][1])
    #print("weights BEFORE STEP----------------\n",model.param()[0][0])
    sgd.step()
    #print("weights AFTER STEP----------------\n",model.param()[0][0])

LOSS ::: tensor(0.5437)
LOSS ::: tensor(0.5089)
LOSS ::: tensor(0.5012)
LOSS ::: tensor(0.4982)
LOSS ::: tensor(0.4965)
LOSS ::: tensor(0.4953)
LOSS ::: tensor(0.4944)
LOSS ::: tensor(0.4937)
LOSS ::: tensor(0.4930)
LOSS ::: tensor(0.4925)
LOSS ::: tensor(0.4920)
LOSS ::: tensor(0.4915)
LOSS ::: tensor(0.4911)
LOSS ::: tensor(0.4906)
LOSS ::: tensor(0.4902)
LOSS ::: tensor(0.4897)
LOSS ::: tensor(0.4893)
LOSS ::: tensor(0.4889)
LOSS ::: tensor(0.4885)
LOSS ::: tensor(0.4881)
LOSS ::: tensor(0.4877)
LOSS ::: tensor(0.4873)
LOSS ::: tensor(0.4869)
LOSS ::: tensor(0.4866)
LOSS ::: tensor(0.4862)
LOSS ::: tensor(0.4859)
LOSS ::: tensor(0.4856)
LOSS ::: tensor(0.4852)
LOSS ::: tensor(0.4849)
LOSS ::: tensor(0.4846)
LOSS ::: tensor(0.4842)
LOSS ::: tensor(0.4839)
LOSS ::: tensor(0.4836)
LOSS ::: tensor(0.4832)
LOSS ::: tensor(0.4829)
LOSS ::: tensor(0.4826)
LOSS ::: tensor(0.4823)
LOSS ::: tensor(0.4819)
LOSS ::: tensor(0.4816)
LOSS ::: tensor(0.4812)
LOSS ::: tensor(0.4809)
LOSS ::: tensor(

In [ ]:
# Modules sequence
model = SS.Sequential(m1,relu,m2,relu,m4,relu)
model.zero_grad()
# loss criterion
MSE = MS.MSE()
cross = MS.CrossEntropyLoss()
#train
for e in range(1000):
    output = model(data)
    loss = MSE.forward(output,target)/1000
    print("LOSS :::",loss)
    dl_dx = MSE.backward(output,target)/1000 
    #dividing dl_dx makes it stable somehow (or just reduce lr)
    model.zero_grad()
    model.backward(dl_dx)
    sgd = OS.SGD(model.param(),eta=9e-2)
    #Must initialize the optimizer here to give it the right parameters (after backward) to update
    sgd.step()